In [1]:
from bs4 import BeautifulSoup as bs
import time
import requests

from selenium import webdriver #splinter does not work so I had to use selenium in order to get the final html since the nasa website usees psudo elements such as ::before and ::after to load the latest news.as

import pandas as pd

#Settings for headless mode.
options = webdriver.ChromeOptions()
options.add_argument('headless')

#path to the driver and load the options.
browser = webdriver.Chrome("/Users/Sebastian/Documents/GitHub/Data Visualization Bootcamp/Sebastian Homework/Web-Scraping-Challenge/chromedriver",chrome_options = options)

marsInfo_dict = {}

In [2]:
#Code to get NASA Mars News

url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&year=2020%3Apublish_date&category=19%2C165%2C184%2C204&blank_scope=Latest"

#Open url.
browser.get(url)

#Time to let the website load all the elements
time.sleep(4)

#save the html source.
html = browser.page_source

#Use bs4 to parse the html response.
soup = bs(html, "html.parser")

#Collect the latest news title
news_title = soup.find_all('li', class_="slide")[0].find(class_="content_title").text

news_p = soup.find_all('li', class_="slide")[0].text

print(news_title)
print("\n")
print(news_p)

marsInfo_dict['news_title'] = news_title
marsInfo_dict['news_p'] = news_p
    

A New Video Captures the Science of NASA's Perseverance Mars Rover


With a targeted launch date of July 30, the next robotic scientist NASA is sending to the to the Red Planet has big ambitions.A New Video Captures the Science of NASA's Perseverance Mars RoverJuly 27, 2020A New Video Captures the Science of NASA's Perseverance Mars RoverWith a targeted launch date of July 30, the next robotic scientist NASA is sending to the to the Red Planet has big ambitions.


In [3]:

#Code to get JPL Mars Space Images - Featured Image

url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

#Opens the url.
browser.get(url)

#Interact with the FULL IMAGE BUTTON
browser.find_element_by_id("full_image").click()

time.sleep(4)

html = browser.page_source

#Use bs4 to parse the html response.
soup = bs(html, "html.parser")

featured_image_url = "https://www.jpl.nasa.gov/" + soup.find_all('img', class_="fancybox-image")[0]['src']

marsInfo_dict['featured_image_url'] = featured_image_url

print(featured_image_url)

https://www.jpl.nasa.gov//spaceimages/images/mediumsize/PIA19685_ip.jpg


In [4]:
#Mars Weather

url = "https://twitter.com/marswxreport?lang=en"
options = webdriver.ChromeOptions()
options.add_argument('headless')

#Open the url.
browser.get(url)

#Time to let the website load all the elements
time.sleep(4)

#save the html source.
html = browser.page_source

#Use bs4 to parse the html response.
soup = bs(html, "html.parser")

mars_weather = soup.find_all('article', class_="css-1dbjc4n r-1loqt21 r-18u37iz r-1ny4l3l r-o7ynqc r-6416eg")[0].text.strip().replace('Mars Weather@MarsWxReport·19hInSight ','')

marsInfo_dict['mars_weather'] = mars_weather

print(mars_weather)

Mars Weather@MarsWxReport·Jul 26InSight sol 591 (2020-07-25) low -91.2ºC (-132.2ºF) high -15.5ºC (4.2ºF)
winds from the WNW at 7.5 m/s (16.9 mph) gusting to 19.0 m/s (42.5 mph)
pressure at 7.90 hPa1029


In [5]:
# Mars Facts

url = 'http://space-facts.com/mars/'

#Load url to pandas read html.
tables = pd.read_html(url)

#Tables
marsFacts_df = tables[0]
earthMars_df = tables[1]

#Rename columns
marsFacts_df.columns = ['Facts', 'Values']


#Outpout
html_outputFacts = marsFacts_df.to_html(index = False)
html_outputFacts = html_outputFacts.replace('\n', '')

html_outputMarsEarth = earthMars_df.to_html(index = False)
html_outputMarsEarth = html_outputMarsEarth.replace('\n', '')

marsInfo_dict['html_outputFacts'] = html_outputFacts
marsInfo_dict['html_outputMarsEarth'] = html_outputMarsEarth


In [6]:
#hemisphereImages
temp_list = []

url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

# start web browser
browser = webdriver.Chrome("/Users/Sebastian/Documents/GitHub/Data Visualization Bootcamp/Sebastian Homework/Web-Scraping-Challenge/chromedriver",chrome_options=options)

#Opens the url.
browser.get(url)

time.sleep(4)

html = browser.page_source

# close web browser
browser.close()

#Use bs4 to parse the html response.
soup = bs(html, "html.parser")

links = soup.find_all('div', class_="description")

for link in links:

    highDef_url = f"https://astrogeology.usgs.gov{link.find('a')['href']}"

    responseHighDef = requests.get(highDef_url)

    soupHighDef = bs(responseHighDef.text, 'html.parser')

    highDef_url = soupHighDef.find_all("div", class_="downloads")[0].find('a')['href']

    title = link.find('h3').text 

    temp_list.append({"title" : title, "img_url" : highDef_url})

marsInfo_dict['hemisphere_image_urls'] = temp_list

In [7]:
marsInfo_dict

{'news_title': "A New Video Captures the Science of NASA's Perseverance Mars Rover",
 'news_p': "With a targeted launch date of July 30, the next robotic scientist NASA is sending to the to the Red Planet has big ambitions.A New Video Captures the Science of NASA's Perseverance Mars RoverJuly 27, 2020A New Video Captures the Science of NASA's Perseverance Mars RoverWith a targeted launch date of July 30, the next robotic scientist NASA is sending to the to the Red Planet has big ambitions.",
 'featured_image_url': 'https://www.jpl.nasa.gov//spaceimages/images/mediumsize/PIA19685_ip.jpg',
 'mars_weather': 'Mars Weather@MarsWxReport·Jul 26InSight sol 591 (2020-07-25) low -91.2ºC (-132.2ºF) high -15.5ºC (4.2ºF)\nwinds from the WNW at 7.5 m/s (16.9 mph) gusting to 19.0 m/s (42.5 mph)\npressure at 7.90 hPa1029',
 'html_outputFacts': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>Facts</th>      <th>Values</th>    </tr>  </thead>  <tbody>    <tr>  